# DenseNet
因为 ResNet 提出了跨层链接的思想，这直接影响了随后出现的卷积网络架构，其中最有名的就是 cvpr 2017 的 best paper，DenseNet。

DenseNet 和 ResNet 不同在于 ResNet 是跨层求和，而 DenseNet 是跨层将特征在通道维度进行拼接，下面可以看看他们两者的图示

![](https://ws4.sinaimg.cn/large/006tNc79ly1fmpvj5vkfhj30uw0anq73.jpg)

![](https://ws1.sinaimg.cn/large/006tNc79ly1fmpvj7fxd1j30vb0eyzqf.jpg)

第一张图是 ResNet，第二张图是 DenseNet，因为是在通道维度进行特征的拼接，所以底层的输出会保留进入所有后面的层，这能够更好的保证梯度的传播，同时能够使用低维的特征和高维的特征进行联合训练，能够得到更好的结果。

DenseNet 主要由 dense block 构成，下面我们来实现一个 densen block

In [ ]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

import tensorflow as tf
from utils import cifar10_input

In [ ]:
batch_size = 64

train_imgs, train_labels, val_imgs, val_labels = cifar10_input.load_data(data_dir='cifar10_data', image_size=96)

In [ ]:
import tensorflow.contrib.slim as slim

### 构建基本卷积单元

In [ ]:
def bn_relu_conv(x, out_depth, scope='dense_basic_conv', reuse=None):
    # 基本卷积单元是: bn->relu-conv
    with tf.variable_scope(scope, reuse=reuse):
        net = slim.batch_norm(x, activation_fn=None, scope='bn')
        net = tf.nn.relu(net, name='activation')
        net = slim.conv2d(net, out_depth, 3, activation_fn=None, normalizer_fn=None, biases_initializer=None, scope='conv')
        
        return net

### 构建`densenet`的基本单元

In [ ]:
def dense_block(x, growth_rate, num_layers, scope='dense_block', reuse=None):
    in_depth = x.get_shape().as_list()[-1]
    
    with tf.variable_scope(scope, reuse=reuse):
        net = x
        for i in range(num_layers):
            out = bn_relu_conv(net, growth_rate, scope='block%d' % i)
            # 将前面所有的输出连接到一起作为下一个基本卷积单元的输入
            net = tf.concat([net, out], axis=-1)
            
        return net

### 构建`transition`层

In [ ]:
def transition(x, out_depth, scope='transition', reuse=None):
    in_depth = x.get_shape().as_list()[-1]
    
    with tf.variable_scope(scope, reuse=reuse):
        net = slim.batch_norm(x, activation_fn=None, scope='bn')
        net = tf.nn.relu(net, name='activation')
        net = slim.conv2d(net, out_depth, 1, activation_fn=None, normalizer_fn=None, biases_initializer=None, scope='conv')
        net = slim.avg_pool2d(net, 2, 2, scope='avg_pool')
        
        return net

### 构建`densenet`整体结构

In [ ]:
def densenet(x, num_classes, growth_rate=32, block_layers=[6, 12, 24, 16], is_training=None, scope='densenet', reuse=None, verbose=False):
    with tf.variable_scope(scope, reuse=reuse):
        with slim.arg_scope([slim.batch_norm], is_training=is_training):
            
            if verbose:
                print('input: {}'.format(x.shape))
            
            with tf.variable_scope('block0'):
                net = slim.conv2d(x, 64, [7, 7], stride=2, normalizer_fn=None, activation_fn=None, scope='conv_7x7')
                net = slim.batch_norm(net, activation_fn=None, scope='bn')
                net = tf.nn.relu(net, name='activation')
                net = slim.max_pool2d(net, [3, 3], stride=2, scope='max_pool')

                if verbose:
                    print('block0: {}'.format(net.shape))

            # 循环创建`dense_block`和`transition`
            for i, num_layers in enumerate(block_layers):
                with tf.variable_scope('block%d' % (i + 1)):
                    net = dense_block(net, growth_rate, num_layers)
                    if i != len(block_layers) - 1:
                        current_depth = net.get_shape().as_list()[-1]
                        net = transition(net, current_depth // 2)

                if verbose:
                    print('block{}: {}'.format(i+1, net.shape))

            with tf.variable_scope('block%d' % (len(block_layers) + 1)):
                net = slim.batch_norm(net, activation_fn=None, scope='bn')
                net = tf.nn.relu(net, name='activation')
                net = tf.reduce_mean(net, [1, 2], name='global_pool', keep_dims=True)

                if verbose:
                    print('block{}: {}'.format(len(block_layers) + 1, net.shape))

            with tf.variable_scope('classification'):
                net = slim.flatten(net, scope='flatten')
                net = slim.fully_connected(net, num_classes, activation_fn=None, normalizer_fn=None, scope='logit')

                if verbose:
                    print('classification: {}'.format(net.shape))

                return net

In [ ]:
with slim.arg_scope([slim.conv2d], activation_fn=tf.nn.relu, normalizer_fn=slim.batch_norm) as sc:
    conv_scope = sc

In [ ]:
is_training = tf.placeholder(tf.bool, name='is_training')

with slim.arg_scope(conv_scope):
    train_out = densenet(train_imgs, 10, is_training=is_training, verbose=True)
    val_out = densenet(val_imgs, 10, is_training=is_training, reuse=True)

### 构建训练

In [ ]:
with tf.variable_scope('loss'):
    train_loss = tf.losses.sparse_softmax_cross_entropy(labels=train_labels, logits=train_out, scope='train')
    val_loss = tf.losses.sparse_softmax_cross_entropy(labels=val_labels, logits=val_out, scope='val')

In [ ]:
with tf.name_scope('accuracy'):
    with tf.name_scope('train'):
        train_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(train_out, axis=-1, output_type=tf.int32), train_labels), tf.float32))
    with tf.name_scope('val'):
        val_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(val_out, axis=-1, output_type=tf.int32), val_labels), tf.float32))

In [ ]:
lr = 0.01

opt = tf.train.MomentumOptimizer(lr, momentum=0.9)

In [ ]:
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = opt.minimize(train_loss)

### 开始训练

In [ ]:
from utils.learning import train_with_bn

In [ ]:
train_with_bn(train_op, train_loss, train_acc, val_loss, val_acc, 20000, is_training)